In [2]:
import csv, requests, os
from bs4 import BeautifulSoup
base_url = 'https://www.nextmanagement.com'
basedir ='nextmanagement.com'
models_details=list()
url = 'https://www.nextmanagement.com/new-york/models?tagged=women'
resp = requests.get(url)

In [5]:
soup = BeautifulSoup(resp.content, 'html5lib')

In [6]:

models_dict = {model.h2.text.strip():base_url+model.a['href'] for model in soup.findAll('article', class_='talent')}

In [12]:

def parse(soup1):
    values = [val.text.split('/')[1] if '/' in val.text else val.text for val in  soup1.find('ul', class_='sidebar_list').findAll('span', class_='value')[:7]]
    features = [ feat.text.upper() for feat in soup1.find('ul', class_='sidebar_list').findAll('h2')[:7]]
    return dict(zip(features, values))

def download_pics(pics):
    for pic in pics:
        path = f"{basedir}/{k}/{pic['alt']}.jpg"
        if not os.path.exists(path):
            with open(path, 'wb') as f:
                f.write(requests.get(pic['src']).content)


def list_to_csv(lst):
    with open(f"{basedir}/models_info.csv",'w') as f:
        c = csv.DictWriter(f, ["NAME","URL", "HEIGHT", "BUST","WAIST","HIPS" ,"EYES","HAIR", "SHOE"])
        c.writeheader()
        for row in lst:
            c.writerow(row)



In [9]:
for k,v in models_dict.items():
    os.makedirs(f"{basedir}/{k}", exist_ok=True)
    print(k,v)
    soup1 = BeautifulSoup(requests.get(v).content, 'html5lib')
    feat_dict= {'NAME':k, 'URL':v}
    feat_dict.update(parse(soup1))
    pics = soup1.find('ul', class_='images').findAll('img')
    models_details.append(feat_dict)
    download_pics(pics)

Abbey Lee https://www.nextmanagement.com/new-york/profile/abbey-lee


In [10]:
list_to_csv(models_details)